### Watertight STL to Node Converter
Convert tetrahedral meshes in stl format into [PyTorch Geometric](https://pytorch-geometric.readthedocs.io/en/latest/) node representation using the [Trimesh](https://trimesh.org/) library. The converter does not work on non-watertight surfaces, i.e. raw_cloud and raw_norm

In [1]:
%pip install torch

Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install numpy

Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
import glob
import torch
from torch_geometric.data import Data
import trimesh
from torch_geometric.data import DataLoader

In [4]:
%pip install chardet

Note: you may need to restart the kernel to use updated packages.


In [5]:
%pip install python-magic
%pip install python-magic-bin

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import magic
file_path = '/Users/rgopalam/Desktop/URAP-GNN/additivemodels/data/raw/rawnorm/0b4f5b9e-8e94-4107-94bf-3c85494ba2e5.stl'
file_type = magic.Magic(mime=True).from_file(file_path)
print(f"File type for rawnorm: {file_type}")


file_path = '/Users/rgopalam/Desktop/URAP-GNN/additivemodels/data/repaired_files/0a7ab8b4-ab18-4524-b8e7-e773ebc6b268.stl00-1.stl'
file_type = magic.Magic(mime=True).from_file(file_path)
print(f"File type for repaired: {file_type}")


File type for rawnorm: application/zip
File type for repaired: application/octet-stream


In [39]:
import zipfile
import os

file_path = '/Users/rgopalam/Desktop/URAP-GNN/additivemodels/data/raw/rawnorm/0b4f5b9e-8e94-4107-94bf-3c85494ba2e5.stl'
extract_dir = '/Users/rgopalam/Desktop/URAP-GNN/additivemodels/data/unzipped/'

# Extract the ZIP file
with zipfile.ZipFile(file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print(f"Files extracted to {extract_dir}")

Files extracted to /Users/rgopalam/Desktop/URAP-GNN/additivemodels/data/unzipped/


In [ ]:
import os
import magic
import pickle

# Path to the unzipped directory
base_dir = '/Users/rgopalam/Desktop/URAP-GNN/additivemodels/data/unzipped/'

# Function to inspect files
def inspect_file(file_path):
    print(f"Inspecting: {file_path}")
    
    # Check file type using libmagic
    file_type = magic.Magic(mime=True).from_file(file_path)
    print(f"File type: {file_type}")
    
    # Handle file based on type
    if file_type == 'text/plain':
        try:
            with open(file_path, 'r') as f:
                print(f"Content (first 500 chars): {f.read(500)}")
        except Exception as e:
            print(f"Error reading text file: {e}")
    elif file_type == 'application/octet-stream':
        try:
            # Attempt to load as pickle
            with open(file_path, 'rb') as f:
                data = pickle.load(f)
            print(f"Pickle file loaded successfully. Data type: {type(data)}")
            print(f"Data preview: {str(data)[:500]}")
        except Exception as e:
            print(f"Error reading pickle file: {e}")
    else:
        print(f"Unhandled file type. File type: {file_type}")
    
    print("-" * 50)

# Walk through the directory and inspect files
for root, dirs, files in os.walk(base_dir):
    for file in files:
        file_path = os.path.join(root, file)
        inspect_file(file_path)

Inspecting: /Users/rgopalam/Desktop/URAP-GNN/additivemodels/data/unzipped/.DS_Store
File type: application/octet-stream
Error reading pickle file: invalid load key, '\x00'.
--------------------------------------------------
Inspecting: /Users/rgopalam/Desktop/URAP-GNN/additivemodels/data/unzipped/0b4f5b9e-8e94-4107-94bf-3c85494ba2e5/version
File type: text/plain
Content (first 500 chars): 3

--------------------------------------------------
Inspecting: /Users/rgopalam/Desktop/URAP-GNN/additivemodels/data/unzipped/0b4f5b9e-8e94-4107-94bf-3c85494ba2e5/data.pkl
File type: application/octet-stream
Error reading pickle file: A load persistent id instruction was encountered,
but no persistent_load function was specified.
--------------------------------------------------
Inspecting: /Users/rgopalam/Desktop/URAP-GNN/additivemodels/data/unzipped/0b4f5b9e-8e94-4107-94bf-3c85494ba2e5/data/0
File type: application/octet-stream
Error reading pickle file: invalid load key, '\x00'.
----------------

In [20]:
import torch

file_path = '/Users/rgopalam/Desktop/URAP-GNN/additivemodels/data/unzipped/0a7ab8b4-ab18-4524-b8e7-e773ebc6b268.stl0-10/data.pkl'

try:
    data = torch.load(file_path)
    print(f"Loaded PyTorch data: {type(data)}")
    print(data)  # Be cautious if the data is large
except Exception as e:
    print(f"Error loading with PyTorch: {e}")


Error loading with PyTorch: A load persistent id instruction was encountered,
but no persistent_load function was specified.


In [21]:
import pickle

file_path = '/Users/rgopalam/Desktop/URAP-GNN/additivemodels/data/unzipped/0a7ab8b4-ab18-4524-b8e7-e773ebc6b268.stl0-10/data.pkl'

def persistent_load(obj):
    print(f"Persistent load called with: {obj}")
    return None  # Stub implementation; real code depends on the context

try:
    with open(file_path, 'rb') as f:
        unpickler = pickle.Unpickler(f)
        unpickler.persistent_load = persistent_load
        data = unpickler.load()
        print(f"Pickle data loaded: {data}")
except Exception as e:
    print(f"Error during manual unpickling: {e}")


Persistent load called with: ('storage', <class 'torch.DoubleStorage'>, '0', 'cpu', 3584)
Error during manual unpickling: 'NoneType' object has no attribute 'dtype'


In [22]:
import torch

def persistent_load_debug(obj):
    print(f"Persistent load called with: {obj}")
    return None  # Stub implementation

file_path = '/Users/rgopalam/Desktop/URAP-GNN/additivemodels/data/unzipped/0a7ab8b4-ab18-4524-b8e7-e773ebc6b268.stl0-10/data.pkl'

try:
    with open(file_path, 'rb') as f:
        data = torch.load(f, map_location='cpu', persistent_load=persistent_load_debug)
        print(f"Data loaded successfully: {data}")
except Exception as e:
    print(f"Error during PyTorch loading: {e}")


Error during PyTorch loading: 'persistent_load' is an invalid keyword argument for load()


In [23]:
%pip install numpy-stl


Note: you may need to restart the kernel to use updated packages.


In [48]:
import os
import pickle
import torch
import numpy as np

# Path to the .pkl file and its directory
file_path = '/Users/rgopalam/Desktop/URAP-GNN/additivemodels/data/unzipped/0b4f5b9e-8e94-4107-94bf-3c85494ba2e5/data.pkl'
directory = os.path.dirname(file_path)

# Persistent load function for unpickling
def persistent_load(obj):
    print(f"Persistent load called with: {obj}")
    if isinstance(obj, tuple) and obj[0] == 'storage':
        storage_type, _, device, size = obj[1:]
        if storage_type == torch.DoubleStorage:
            return torch.empty(size, dtype=torch.float64, device=device).storage()
    raise pickle.UnpicklingError(f"Unknown persistent_load object: {obj}")

# Load and inspect the .pkl file
try:
    with open(file_path, 'rb') as f:
        unpickler = pickle.Unpickler(f)
        unpickler.persistent_load = persistent_load
        data = unpickler.load()
        print("\n--- Successfully Loaded .pkl File ---")
        print(f"Data Type: {type(data)}")
        if isinstance(data, dict):
            print(f"Keys in Dictionary: {data.keys()}")
        elif isinstance(data, list):
            print(f"Length of List: {len(data)}")
        elif isinstance(data, tuple):
            print(f"Length of Tuple: {len(data)}")
            for i, element in enumerate(data):
                print(f"Element {i}: Type: {type(element)}, Shape: {getattr(element, 'shape', 'N/A')}")
except Exception as e:
    print(f"Error loading .pkl file: {e}")

# Inspect files in the same directory
print("\n--- Inspecting Other Files in the Same Directory ---")
for filename in os.listdir(directory):
    file_path = os.path.join(directory, filename)
    print(f"File: {filename}, Size: {os.path.getsize(file_path)} bytes")
    # If the file is another .pkl, try inspecting it
    if filename.endswith('.pkl') and filename != os.path.basename(file_path):
        try:
            with open(file_path, 'rb') as f:
                other_data = pickle.load(f)
                print(f"  - {filename}: Successfully loaded (Type: {type(other_data)})")
                if isinstance(other_data, dict):
                    print(f"    Keys: {other_data.keys()}")
                elif isinstance(other_data, list):
                    print(f"    Length: {len(other_data)}")
        except Exception as e:
            print(f"  - {filename}: Error loading file ({e})")


Persistent load called with: ('storage', <class 'torch.DoubleStorage'>, '0', 'cpu', 3584)
Persistent load called with: ('storage', <class 'torch.DoubleStorage'>, '1', 'cpu', 512)

--- Successfully Loaded .pkl File ---
Data Type: <class 'tuple'>
Length of Tuple: 2
Element 0: Type: <class 'torch.Tensor'>, Shape: torch.Size([512, 7])
Element 1: Type: <class 'torch.Tensor'>, Shape: torch.Size([512])

--- Inspecting Other Files in the Same Directory ---
File: version, Size: 2 bytes
File: data.pkl, Size: 211 bytes
File: data, Size: 128 bytes


In [49]:
import os
import pickle
import magic

# Directory path
directory = '/Users/rgopalam/Desktop/URAP-GNN/additivemodels/data/unzipped/0b4f5b9e-8e94-4107-94bf-3c85494ba2e5'

# Inspect each file in the directory
print("\n--- Inspecting Other Files in the Directory ---")
for filename in os.listdir(directory):
    file_path = os.path.join(directory, filename)
    file_size = os.path.getsize(file_path)
    print(f"\nFile: {filename}, Size: {file_size} bytes")

    # Use magic to determine file type
    try:
        mime = magic.Magic(mime=True)
        file_type = mime.from_file(file_path)
        print(f"File Type: {file_type}")
    except Exception as e:
        print(f"Error determining file type: {e}")
        file_type = None

    # Attempt to open .pkl files
    if filename.endswith('.pkl'):
        try:
            with open(file_path, 'rb') as f:
                content = pickle.load(f)
                print(f"Successfully unpickled {filename}.")
                print(f"Type of content: {type(content)}")
                if isinstance(content, dict):
                    print(f"Keys in dictionary: {content.keys()}")
                elif isinstance(content, list):
                    print(f"List length: {len(content)}")
                elif isinstance(content, tuple):
                    print(f"Tuple length: {len(content)}")
        except Exception as e:
            print(f"Error unpickling file {filename}: {e}")

    # Attempt to read text files
    elif file_type and "text" in file_type:
        try:
            with open(file_path, 'r') as f:
                content = f.read()
                print(f"Text Content of {filename}:\n{content[:500]}")  # Limit to first 500 chars
        except Exception as e:
            print(f"Error reading text file {filename}: {e}")

    # Log if the file type is unsupported
    else:
        print(f"Skipping unsupported or binary file: {filename}")



--- Inspecting Other Files in the Directory ---

File: version, Size: 2 bytes
File Type: text/plain
Text Content of version:
3


File: data.pkl, Size: 211 bytes
File Type: application/octet-stream
Error unpickling file data.pkl: A load persistent id instruction was encountered,
but no persistent_load function was specified.

File: data, Size: 128 bytes
Error determining file type: [Errno 21] Is a directory: '/Users/rgopalam/Desktop/URAP-GNN/additivemodels/data/unzipped/0b4f5b9e-8e94-4107-94bf-3c85494ba2e5/data'
Skipping unsupported or binary file: data


In [50]:
import os
import pickle
import magic
import torch

# Paths to the files and directory
main_pkl_path = '/Users/rgopalam/Desktop/URAP-GNN/additivemodels/data/unzipped/0b4f5b9e-8e94-4107-94bf-3c85494ba2e5/data.pkl'
data_dir_path = '/Users/rgopalam/Desktop/URAP-GNN/additivemodels/data/unzipped/0b4f5b9e-8e94-4107-94bf-3c85494ba2e5/data'

# Persistent load function for unpickling
def persistent_load(obj):
    print(f"Persistent load called with: {obj}")
    if isinstance(obj, tuple) and obj[0] == 'storage':
        storage_type, _, device, size = obj[1:]
        if storage_type == torch.DoubleStorage:
            return torch.empty(size, dtype=torch.float64, device=device).storage()
    raise pickle.UnpicklingError(f"Unknown persistent_load object: {obj}")

# Step 1: Load and inspect main data.pkl
print("\n--- Step 1: Loading Main data.pkl ---")
try:
    with open(main_pkl_path, 'rb') as f:
        unpickler = pickle.Unpickler(f)
        unpickler.persistent_load = persistent_load
        main_data = unpickler.load()
        print("\n--- Successfully Loaded Main data.pkl ---")
        print(f"Data Type: {type(main_data)}")
        if isinstance(main_data, dict):
            print(f"Keys: {main_data.keys()}")
        elif isinstance(main_data, tuple):
            print(f"Tuple Length: {len(main_data)}")
        elif isinstance(main_data, list):
            print(f"List Length: {len(main_data)}")
except Exception as e:
    print(f"Error loading main data.pkl: {e}")

# Step 2: Inspect the 'data.pkl' file in the same directory
print("\n--- Step 2: Inspecting Additional data.pkl ---")
try:
    with open(main_pkl_path, 'rb') as f:
        unpickler = pickle.Unpickler(f)
        unpickler.persistent_load = persistent_load
        data_pkl_content = unpickler.load()
        print("\n--- Successfully Loaded data.pkl ---")
        print(f"Data Type: {type(data_pkl_content)}")
        if isinstance(data_pkl_content, dict):
            print(f"Keys: {data_pkl_content.keys()}")
        elif isinstance(data_pkl_content, tuple):
            print(f"Tuple Length: {len(data_pkl_content)}")
        elif isinstance(data_pkl_content, list):
            print(f"List Length: {len(data_pkl_content)}")
except Exception as e:
    print(f"Error loading additional data.pkl: {e}")

# Step 3: Inspect the 'data' directory and its contents
print("\n--- Step 3: Inspecting Contents of the 'data' Directory ---")
try:
    for filename in os.listdir(data_dir_path):
        file_path = os.path.join(data_dir_path, filename)
        if os.path.isfile(file_path):
            print(f"\nFile: {filename}, Size: {os.path.getsize(file_path)} bytes")
            try:
                # Determine file type using python-magic
                mime = magic.Magic(mime=True)
                file_type = mime.from_file(file_path)
                print(f"File Type: {file_type}")

                # If it's a pickle file, try to unpickle it
                if filename.endswith('.pkl'):
                    try:
                        with open(file_path, 'rb') as f:
                            content = pickle.load(f)
                            print(f"Successfully unpickled {filename}.")
                            print(f"Type of content: {type(content)}")
                            if isinstance(content, dict):
                                print(f"Keys: {content.keys()}")
                            elif isinstance(content, tuple):
                                print(f"Tuple Length: {len(content)}")
                            elif isinstance(content, list):
                                print(f"List Length: {len(content)}")
                    except Exception as e:
                        print(f"Error unpickling file {filename}: {e}")
                # If it's a text file, read the content
                elif "text" in file_type:
                    try:
                        with open(file_path, 'r') as f:
                            print(f"Text Content of {filename}:\n{f.read()[:500]}")
                    except Exception as e:
                        print(f"Error reading text file {filename}: {e}")
            except Exception as e:
                print(f"Error determining file type for {filename}: {e}")
        elif os.path.isdir(file_path):
            print(f"Directory: {filename}")
except Exception as e:
    print(f"Error inspecting files in 'data' directory: {e}")



--- Step 1: Loading Main data.pkl ---
Persistent load called with: ('storage', <class 'torch.DoubleStorage'>, '0', 'cpu', 3584)
Persistent load called with: ('storage', <class 'torch.DoubleStorage'>, '1', 'cpu', 512)

--- Successfully Loaded Main data.pkl ---
Data Type: <class 'tuple'>
Tuple Length: 2

--- Step 2: Inspecting Additional data.pkl ---
Persistent load called with: ('storage', <class 'torch.DoubleStorage'>, '0', 'cpu', 3584)
Persistent load called with: ('storage', <class 'torch.DoubleStorage'>, '1', 'cpu', 512)

--- Successfully Loaded data.pkl ---
Data Type: <class 'tuple'>
Tuple Length: 2

--- Step 3: Inspecting Contents of the 'data' Directory ---

File: 0, Size: 28672 bytes
File Type: application/octet-stream

File: 1, Size: 4096 bytes
File Type: application/octet-stream


In [51]:
import struct

# Paths to binary files
binary_files = [
    os.path.join(data_dir_path, '0'),
    os.path.join(data_dir_path, '1'),
]

print("\n--- Inspecting Octet-Stream Binary Files ---")
for file_path in binary_files:
    print(f"\nFile: {os.path.basename(file_path)}, Size: {os.path.getsize(file_path)} bytes")

    # Attempt to detect file format with magic
    try:
        file_type = magic.Magic(mime=True).from_file(file_path)
        print(f"File Type Detected: {file_type}")
    except Exception as e:
        print(f"Error detecting file type: {e}")

    # Try opening as a pickle file
    try:
        with open(file_path, 'rb') as f:
            content = pickle.load(f)
            print(f"Successfully unpickled {os.path.basename(file_path)}.")
            print(f"Type of content: {type(content)}")
            if isinstance(content, dict):
                print(f"Keys: {content.keys()}")
            elif isinstance(content, tuple):
                print(f"Tuple Length: {len(content)}")
    except Exception as e:
        print(f"Error unpickling file {os.path.basename(file_path)}: {e}")

    # Try reading first 64 bytes for hex/ASCII preview
    try:
        with open(file_path, 'rb') as f:
            preview = f.read(64)
            print("Hex Preview:", preview.hex())
            print("ASCII Preview:", preview.decode(errors='replace'))
    except Exception as e:
        print(f"Error reading file {os.path.basename(file_path)}: {e}")



--- Inspecting Octet-Stream Binary Files ---

File: 0, Size: 28672 bytes
File Type Detected: application/octet-stream
Error unpickling file 0: invalid load key, '\x00'.
Hex Preview: 00000040d9a4febfabaaaaaa5347db3f000000c0f3efc63f000000000000f0bf0000000000000000000000000000000000807f3832f1df3f00000040d9a4febf
ASCII Preview:    @٤������SG�?   ����?      �                 �82��?   @٤��

File: 1, Size: 4096 bytes
File Type Detected: application/octet-stream
Error unpickling file 1: invalid load key, '\x00'.
Hex Preview: 00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000
ASCII Preview:                                                                 


In [52]:
from stl import mesh
import trimesh

# Path to the file
file_path = '/Users/rgopalam/Desktop/URAP-GNN/additivemodels/data/unzipped/0b4f5b9e-8e94-4107-94bf-3c85494ba2e5/data/0'

print("\n--- Testing if File is an STL ---")

# Test with numpy-stl
try:
    your_mesh = mesh.Mesh.from_file(file_path)
    print("Successfully loaded as STL with numpy-stl.")
    print(f"Number of triangles: {len(your_mesh)}")
except Exception as e:
    print(f"Error loading with numpy-stl: {e}")

# Test with trimesh
try:
    trimesh_mesh = trimesh.load(file_path, file_type='stl')
    print("Successfully loaded as STL with trimesh.")
    print(f"Vertices: {len(trimesh_mesh.vertices)}, Faces: {len(trimesh_mesh.faces)}")
    trimesh_mesh.show()
except Exception as e:
    print(f"Error loading with trimesh: {e}")



--- Testing if File is an STL ---
Successfully loaded as STL with numpy-stl.
Number of triangles: 0
Error loading with trimesh: decode() argument 'encoding' must be str, not None


In [1]:
from stl import mesh
import trimesh

# Path to the binary file
file_path = '/Users/rgopalam/Desktop/URAP-GNN/additivemodels/data/unzipped/0b4f5b9e-8e94-4107-94bf-3c85494ba2e5/data/0'

print("\n--- Step 1: Attempting to Load with numpy-stl ---")
# Test with numpy-stl and inspect data
try:
    your_mesh = mesh.Mesh.from_file(file_path)
    print("Successfully loaded as STL with numpy-stl.")
    print(f"Number of triangles: {len(your_mesh)}")
    if len(your_mesh) > 0:
        print("Triangle Data Preview:")
        print(your_mesh.vectors[:5])  # Print the first 5 triangle vectors
    else:
        print("The STL file appears to be empty (no triangles).")
except Exception as e:
    print(f"Error loading with numpy-stl: {e}")

print("\n--- Step 2: Reading STL Header ---")
# Read the first 80 bytes of the binary file (STL header)
try:
    with open(file_path, 'rb') as f:
        header = f.read(80)
        print("--- STL Header ---")
        print(header.decode(errors='replace'))  # Decode as text (replace invalid characters)
        print("Hex:", header.hex())  # Display hex representation of the header
except Exception as e:
    print(f"Error reading STL header: {e}")

print("\n--- Step 3: Attempting to Load with trimesh ---")
# Test with trimesh
try:
    trimesh_mesh = trimesh.load(file_path, file_type='stl', encoding='binary')  # Force binary format
    print("Successfully loaded as STL with trimesh.")
    print(f"Vertices: {len(trimesh_mesh.vertices)}, Faces: {len(trimesh_mesh.faces)}")
    if len(trimesh_mesh.vertices) > 0 and len(trimesh_mesh.faces) > 0:
        print("Displaying mesh using trimesh...")
        trimesh_mesh.show()
    else:
        print("The STL file contains no valid geometry.")
except Exception as e:
    print(f"Error loading with trimesh: {e}")



--- Step 1: Attempting to Load with numpy-stl ---
Successfully loaded as STL with numpy-stl.
Number of triangles: 0
The STL file appears to be empty (no triangles).

--- Step 2: Reading STL Header ---
--- STL Header ---
   @٤������SG�?   ����?      �                 �82��?   @٤������i�?   ����?
Hex: 00000040d9a4febfabaaaaaa5347db3f000000c0f3efc63f000000000000f0bf0000000000000000000000000000000000807f3832f1df3f00000040d9a4febfabaaaaea1869f83f000000e080f9d03f

--- Step 3: Attempting to Load with trimesh ---
Error loading with trimesh: decode() argument 'encoding' must be str, not None
